In [ ]:
# Mounting Google Drive locally
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
pip install pyradiomics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 26.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Discarding https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz (from https://pypi.org/simple/pyradiomics/): Requested pyradiomics from https://files.pythonhosted.org/packages/03/c1/20fc2c50ab1e3304da36d866042a1905a2b05a1431ece35448ab6b4578f2/pyradiomics-3.1.0.tar.gz has inconsistent version: expected '3.1.0', but metadata has '3.0.1a1'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 19.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.8/117.8 kB 13.8 MB/s eta 0:00:

In [ ]:
import os
import pandas as pd
from radiomics import featureextractor
import re

In [ ]:
extractor = featureextractor.RadiomicsFeatureExtractor()

INFO:radiomics.featureextractor:No valid config parameter, using defaults: {'minimumROIDimensions': 2, 'minimumROISize': None, 'normalize': False, 'normalizeScale': 1, 'removeOutliers': None, 'resampledPixelSpacing': None, 'interpolator': 'sitkBSpline', 'preCrop': False, 'padDistance': 5, 'distances': [1], 'force2D': False, 'force2Ddimension': 0, 'resegmentRange': None, 'label': 1, 'additionalInfo': True}
INFO:radiomics.featureextractor:Enabled image types: {'Original': {}}
INFO:radiomics.featureextractor:Enabled features: {'firstorder': [], 'glcm': [], 'gldm': [], 'glrlm': [], 'glszm': [], 'ngtdm': [], 'shape': []}


In [ ]:
# Regex patterns to match the T1 modality images and GlisterBoost output labels
# Match both 'HF' and '900-00-' prefixes
t1_pattern = re.compile(r'(HF\d+|900-00-\d{4}).*_t1_LPS_rSRI.nii.gz')
segmentation_pattern = re.compile(r'(HF\d+|900-00-\d{4}).*_GlistrBoost_out-labels.nii')

# Directory where the patient folders are located
rootDir = '/content/drive/MyDrive/final_project/Rembrandt_data'

# Initialize a list to store the data of each patient
data = []

# Walk through the directory and process each patient's data
for dirName, subdirList, fileList in os.walk(rootDir):
    t1_image_path = None
    segmentation_path = None

    # Extract patient ID based on the directory structure
    patient_id = re.search(r'(HF\d+|900-00-\d{4})', dirName)
    if patient_id:
        patient_id = patient_id.group(0)

        # Find the relevant files for the current patient
        for fname in fileList:
            if t1_pattern.match(fname):
                t1_image_path = os.path.join(dirName, fname)
                print(f"Found T1 image: {t1_image_path}")
            elif segmentation_pattern.match(fname):
                segmentation_path = os.path.join(dirName, fname)
                print(f"Found segmentation file: {segmentation_path}")

        # If both the T1 image and segmentation files are found, extract features
        if t1_image_path and segmentation_path:
            try:
                print(f"Extracting features from image: {t1_image_path}")
                features = extractor.execute(t1_image_path, segmentation_path)
                features['PatientID'] = patient_id
                data.append(features)
                print(f"Features extracted for {patient_id}")
            except Exception as e:
                print(f"Failed to process {patient_id}: {e}")

# After extracting features and storing them in 'data', create a DataFrame
all_features = pd.DataFrame(data)

# Define the output directory path and create it if it doesn't exist
output_dir = os.path.join(rootDir, 'output')
os.makedirs(output_dir, exist_ok=True)

# Define the full path for the CSV file and save it
output_csv_path = os.path.join(output_dir, 'feature_matrix.csv')
all_features.to_csv(output_csv_path, index=False)
print(f"Feature extraction complete. The feature matrix is saved to {output_csv_path}.")


INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


Found segmentation file: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_41_50/HF1409_1994.09.23/HF1409_1994.09.23_GlistrBoost_out-labels.nii
Found T1 image: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_41_50/HF1409_1994.09.23/HF1409_1994.09.23_t1_LPS_rSRI.nii.gz
Extracting features from image: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_41_50/HF1409_1994.09.23/HF1409_1994.09.23_t1_LPS_rSRI.nii.gz


INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


Features extracted for HF1409
Found segmentation file: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_41_50/HF1458_1994.12.28/HF1458_1994.12.28_GlistrBoost_out-labels.nii
Found T1 image: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_41_50/HF1458_1994.12.28/HF1458_1994.12.28_t1_LPS_rSRI.nii.gz
Extracting features from image: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_41_50/HF1458_1994.12.28/HF1458_1994.12.28_t1_LPS_rSRI.nii.gz


INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


...

Features extracted for 900-00-5404
Found T1 image: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_11_20/900-00-5459_2005.11.21/900-00-5459_2005.11.21_t1_LPS_rSRI.nii.gz
Found segmentation file: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_11_20/900-00-5459_2005.11.21/900-00-5459_2005.11.21_GlistrBoost_out-labels.nii
Extracting features from image: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_11_20/900-00-5459_2005.11.21/900-00-5459_2005.11.21_t1_LPS_rSRI.nii.gz


INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm
INFO:radiomics.featureextractor:Calculating features with label: 1
INFO:radiomics.featureextractor:Loading image and mask


Features extracted for 900-00-5459
Found segmentation file: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_11_20/900-00-5396_2005.08.13/900-00-5396_2005.08.13_GlistrBoost_out-labels.nii
Found T1 image: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_11_20/900-00-5396_2005.08.13/900-00-5396_2005.08.13_t1_LPS_rSRI.nii.gz
Extracting features from image: /content/drive/MyDrive/final_project/Rembrandt_data/Rembrandt_Final_Data/Patients_11_20/900-00-5396_2005.08.13/900-00-5396_2005.08.13_t1_LPS_rSRI.nii.gz


INFO:radiomics.featureextractor:Computing shape
INFO:radiomics.featureextractor:Adding image type "Original" with custom settings: {}
INFO:radiomics.featureextractor:Calculating features for original image
INFO:radiomics.featureextractor:Computing firstorder
INFO:radiomics.featureextractor:Computing glcm
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
INFO:radiomics.featureextractor:Computing gldm
INFO:radiomics.featureextractor:Computing glrlm
INFO:radiomics.featureextractor:Computing glszm
INFO:radiomics.featureextractor:Computing ngtdm


Features extracted for 900-00-5396
Feature extraction complete. The feature matrix is saved to /content/drive/MyDrive/final_project/Rembrandt_data/output/feature_matrix.csv.


Dont use this code

In [ ]:
#   Dont use this code

# Regex pattern to match the T1 modality images and GlisterBoost output labels


t1_pattern = re.compile(r'(HF\d+|900-00-\d{4}).*_t1_LPS_rSRI.nii.gz')
segmentation_pattern = re.compile(r'(HF\d+|900-00-\d{4}).*_GlistrBoost_out-labels.nii')

# Directory where the patient folders are located
rootDir = '/content/drive/MyDrive/7009 Final Project/Rembrandt_data'

# Initialize a list to store the data of each patient
data = []

# Walk through the directory and process each patient's data
for dirName, subdirList, fileList in os.walk(rootDir):
    t1_image_path = None
    segmentation_path = None

    # Extract patient ID based on the directory structure
    patient_id = os.path.basename(dirName)

    # Find the relevant files for the current patient
    for fname in fileList:
        if t1_pattern.match(fname):
            t1_image_path = os.path.join(dirName, fname)
            print(f"Found T1 image: {t1_image_path}")
        elif segmentation_pattern.match(fname):
            segmentation_path = os.path.join(dirName, fname)
            print(f"Found segmentation file: {segmentation_path}")

    # If both the T1 image and segmentation files are found, extract features
    if t1_image_path and segmentation_path:
        try:
            print(f"Extracting features from image: {t1_image_path}")
            features = extractor.execute(t1_image_path, segmentation_path)
            features['PatientID'] = patient_id
            data.append(features)
            print(f"Features extracted for {patient_id}")
        except Exception as e:
            print(f"Failed to process {patient_id}: {e}")

# After extracting features and storing them in 'data', create a DataFrame
all_features = pd.DataFrame(data)

# Define the output directory path and create it if it doesn't exist
output_dir = os.path.join(rootDir, 'output')
os.makedirs(output_dir, exist_ok=True)

# Define the full path for the CSV file and save it
output_csv_path = os.path.join(output_dir, 'feature_matrix.csv')
all_features.to_csv(output_csv_path, index=False)
print(f"Feature extraction complete. The feature matrix is saved to {output_csv_path}.")